Copying code from https://google.github.io/mediapipe/solutions/pose#python-solution-api


In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame()

In [16]:
PoseData = list()

In [13]:
# For webcam input:
from ast import ListComp


cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # prinitng landmarks
    print(results.pose_landmarks.landmark[1])

    # PoseData.append({four.})
    

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

x: 0.5902385711669922
y: 0.4788278043270111
z: -0.5590301752090454
visibility: 0.9998564720153809

x: 0.5896762013435364
y: 0.48093774914741516
z: -0.5322672724723816
visibility: 0.99981689453125

x: 0.5888206362724304
y: 0.4810924828052521
z: -0.5255184769630432
visibility: 0.999718964099884

x: 0.5884302258491516
y: 0.48116302490234375
z: -0.5295531153678894
visibility: 0.999726414680481

x: 0.5882575511932373
y: 0.4799160361289978
z: -0.5255409479141235
visibility: 0.9997051358222961

x: 0.5872222781181335
y: 0.4806155860424042
z: -0.5171496868133545
visibility: 0.9997171759605408

x: 0.5872542262077332
y: 0.47993481159210205
z: -0.4606808125972748
visibility: 0.9997193813323975

x: 0.5868395566940308
y: 0.48128098249435425
z: -0.47278454899787903
visibility: 0.9997243881225586

x: 0.5866928696632385
y: 0.4808315634727478
z: -0.4778250753879547
visibility: 0.9997310042381287

x: 0.5865364074707031
y: 0.48079749941825867
z: -0.4338042438030243
visibility: 0.9997333884239197

x: 0.586

KeyboardInterrupt: 